# Assignment 3 on Natural Language Processing

## Date : 30th Sept, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.



Please submit with outputs. 

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving IMDBDataset.csv to IMDBDataset.csv


In [ ]:
data = pd.read_csv("IMDBDataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

TAG_RE = re.compile(r'<[^>]+>')
sw = set(stopwords.words('english')) 
ps = PorterStemmer() 
sum = 0 
cnt = 0
ma = 0
vocab = set() 
poscnt = 0
negcnt = 0

for index,row in data.iterrows(): 
  txt = row['review']
  tmp = row['sentiment']
  txt1 = TAG_RE.sub('', txt)                       #remove html
  txt2 = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt1)
  #remove url
  txt3 = re.sub(r'[^A-Za-z0-9 ]+', '', txt2)       #remove non alhpa-numeric
  txt4 = word_tokenize(txt3)                       #remove stopwords
  txt4 = [w for w in txt4 if w not in sw]  
  txt5 = []                                        #stemming
  for w in txt4:
    txt5.append(ps.stem(w))
    vocab.add(w)
  if tmp == 'positive':
    poscnt += len(txt5)
  else:
    negcnt += len(txt5)
  sum = sum+len(txt5)
  cnt = cnt+1
  ma = max(ma,len(txt5))
  row['review'] = " ".join(txt5)
  if index<9:
    print(row['review'])
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
one review mention watch 1 Oz episod youll hook right exactli happen meth first thing struck Oz brutal unflinch scene violenc set right word GO trust show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call OZ nicknam given oswald maximum secur state penitentari It focu mainli emerald citi experi section prison cell glass front face inward privaci high agenda Em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod I ever saw struck nasti surreal I couldnt say I readi I watch 

In [ ]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
print('Avg length = ', sum/cnt)
print('Max length of sentence: ',ma)
print('Size of vocabulary ',len(vocab))
print('Number of words in positive class',poscnt)
print('Number of words in negative class',negcnt)

Avg length =  124.58464
Max length of sentence:  1444
Size of vocabulary  182686
Number of words in positive class 3144844
Number of words in negative class 3084388


# Naive Bayes classifier

In [ ]:
# get reviews column from df
reviews = data['review']

# get labels column from df
labels = data['sentiment']

In [ ]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
#print(encoded_labels)
# print(enc.classes_)

In [ ]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.
# train_sentences, test_sentences, train_labels, test_labels
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['review'], encoded_labels,test_size=0.2,stratify=encoded_labels )
print(len(X_train))
print(len(X_test))

40000
10000


Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words

X_train_pos = []
X_train_neg = []
i = 0
for row in X_train:
  if y_train[i]==1:
    X_train_pos.append(row)
  else:
    X_train_neg.append(row)
  i += 1
#max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
vec1 = CountVectorizer(max_features = 3000)
XP = vec1.fit_transform(X_train_pos)

vec2 = CountVectorizer(max_features = 3000)
XN = vec2.fit_transform(X_train_neg)


In [ ]:
# Use laplace smoothing for words in test set not present in vocab of train set
word_list = vec1.get_feature_names()
count_list = XP.toarray().sum(axis=0)
freq_pos = dict(zip(word_list,count_list))
print(freq_pos)

word_list1 = vec2.get_feature_names()
count_list1 = XN.toarray().sum(axis=0)
freq_neg = dict(zip(word_list1,count_list1))
print(freq_neg)

pos_cnt = count_list.sum(axis = 0)
neg_cnt = count_list1.sum(axis = 0)

print(pos_cnt)
print(neg_cnt)
prob_unk_pos = 1.0/(pos_cnt+3000.0)
prob_unk_neg = 1.0/(neg_cnt+3000.0)
print(prob_unk_pos)
print(prob_unk_neg)

{'10': 1653, '100': 308, '1010': 292, '11': 165, '12': 335, '13': 137, '14': 131, '15': 299, '18': 108, '1930': 188, '1940': 158, '1950': 267, '1960': 146, '1970': 230, '1980': 243, '1990': 135, '20': 458, '2000': 111, '2001': 163, '2005': 135, '2006': 117, '20th': 132, '25': 130, '30': 365, '40': 305, '50': 363, '60': 356, '70': 573, '710': 209, '80': 677, '810': 249, '90': 327, '910': 187, 'abandon': 212, 'abc': 118, 'abil': 475, 'abl': 1141, 'absolut': 1278, 'absorb': 137, 'absurd': 182, 'abu': 340, 'academi': 305, 'accent': 367, 'accept': 702, 'access': 126, 'accid': 461, 'accompani': 204, 'accomplish': 236, 'accord': 196, 'account': 233, 'accu': 188, 'accur': 342, 'achiev': 558, 'across': 813, 'act': 5306, 'action': 2699, 'activ': 233, 'actor': 4843, 'actress': 1213, 'actual': 3201, 'ad': 638, 'adam': 290, 'adapt': 676, 'add': 968, 'addict': 253, 'addit': 408, 'address': 149, 'admir': 417, 'admit': 550, 'adopt': 148, 'ador': 263, 'adult': 755, 'advanc': 228, 'advantag': 161, 'adve

In [ ]:
# Build the model. Don't use the model from sklearn
import math
def classify(sent):
  pos_prob = 1.0
  neg_prob = 1.0
  txt = word_tokenize(sent)
  for w in txt:
    if w in freq_pos.keys():
      pos_prob += math.log(1+freq_pos[w])
      pos_prob -= math.log(pos_cnt+3000.0)
    else:
      pos_prob += math.log(prob_unk_pos)
    if w in freq_neg.keys():
      neg_prob += math.log(1+freq_neg[w])
      neg_prob -= math.log(neg_cnt+3000.0)
    else:
      neg_prob += math.log(prob_unk_neg)
  if pos_prob > neg_prob:
    return 1
  else:
    return 0


In [ ]:
# Test the model on test set and report Accuracy
n = 0
d = 0
i = 0
for row in X_test:
  tmp = classify(row)
  if tmp == y_test[i]:
    n += 1
  d += 1
  i += 1
print('Accuaracy of the model is ',n/d)

Accuaracy of the model is  0.8123


# *LSTM* based Classifier

Use the above train and test splits.

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 150 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [ ]:
# tokenize sentences
train_sentences = X_train
test_sentences = X_test
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [ ]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 100)          1800000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_4 (Dense)              (None, 24)                3096      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 1,887,601
Trainable params: 1,887,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 5
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 194s 173ms/step - loss: 0.3518 - accuracy: 0.8481 - val_loss: 0.2909 - val_accuracy: 0.8770
Epoch 2/5
1125/1125 [==============================] - 197s 175ms/step - loss: 0.2123 - accuracy: 0.9197 - val_loss: 0.3319 - val_accuracy: 0.8740
Epoch 3/5
1125/1125 [==============================] - 194s 173ms/step - loss: 0.1477 - accuracy: 0.9462 - val_loss: 0.4363 - val_accuracy: 0.8702
Epoch 4/5
1125/1125 [==============================] - 197s 175ms/step - loss: 0.0993 - accuracy: 0.9657 - val_loss: 0.4462 - val_accuracy: 0.8520
Epoch 5/5
1125/1125 [==============================] - 196s 174ms/step - loss: 0.0704 - accuracy: 0.9766 - val_loss: 0.4412 - val_accuracy: 0.8585


In [ ]:
# Calculate accuracy on Test data

prediction = model.predict(test_padded)

# Get probabilities
y_pred = (prediction>0.5)

# Get labels based on probability 1 if p>= 0.5 else 0

print(classification_report(y_test, y_pred))

# Accuracy : one can use classification_report from sklearn

              precision    recall  f1-score   support

           0       0.90      0.83      0.86      5000
           1       0.84      0.91      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## Get predictions for random examples

In [ ]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences =  tokenizer.texts_to_sequences(sentence)


# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length) 

# Get probabilities

#pred = (model.predict(padded)>0.5)

# Get labels based on probability 1 if p>= 0.5 else 0

for i in pred:
  if i:
    print('Guessed positive')
  else:
    print('Guesses negative')


Guessed positive
Guesses negative
Guesses negative
